# Explore data

Simple notebook to explore the dataset

In [52]:
import numpy as np
import matplotlib.pyplot as plt
from gwpy.table import EventTable

## Load in the selected detected events:

In [23]:
# Only include the true variable names in the code
COLUMN_NAMES = ["simulation_id", 
                "longitude", 
                "latitude", 
                "inclination", 
                "distance", 
                "mass1", 
                "mass2", 
                "spin1z", 
                "spin2z", 
                "polarization", 
                "coa_phase", 
                "geocent_end_time", 
                "geocent_end_time_ns"]

print(f"Number of columns: {len(COLUMN_NAMES)}")

Number of columns: 13


In [25]:
data_dir = "./data_split/"
bns_o5 = data_dir + "bns_O5HL_injections.dat"

# Read it
data = np.genfromtxt(bns_o5, names = True)
data = np.array([list(row) for row in data])
data = data.T # (n_dim, n_events)

# Also make a dict version
data_dict = {name: data[i] for i, name in enumerate(COLUMN_NAMES)}

Example

In [26]:
example = data[:, 0]
for key, val in zip(COLUMN_NAMES, example):
    print(f"{key}: {val}")

simulation_id: 9.0
longitude: 3.7971859
latitude: 0.87535684
inclination: 0.38678959
distance: 203.77752
mass1: 2.0644817
mass2: 1.6678797
spin1z: -0.061078237
spin2z: -0.00031229967
polarization: 1.3379265069961548
coa_phase: -2.081282138824463
geocent_end_time: 1026305955.0
geocent_end_time_ns: 487770796.0


## Need the SNR

In [27]:
xml_filename = "/home/thibeau.wouters/gw-datasets/H0_inference_O5/events_O5.xml"

By hand:

In [60]:
table = EventTable.read(xml_filename, tablename = "coinc_inspiral")

# Get the keys of this table
keys = table.keys()
print("keys")
print(keys)

# Get some values that I want to use

simulation_id_xml = np.array(table["coinc_event_id"])
snr_xml = np.array(table["snr"])
end_time = np.array(table["end_time"])


# Show the SNR of the above example
idx = np.where(simulation_id_xml == int(example[0]))[0][0]
snr_example = snr_xml[idx]
end_time_example = end_time[idx]

print("snr_example")
print(snr_example)

print("end_time_example")
print(end_time_example)

keys
['coinc_event_id', 'combined_far', 'end_time', 'end_time_ns', 'false_alarm_rate', 'ifos', 'mass', 'mchirp', 'minimum_duration', 'snr']
snr_example
20.71345431810761
end_time_example
1026305955


This seems to be working fine, now let us put into a complete function

In [61]:
def get_snr_array(table: EventTable, simulation_ids: np.ndarray) -> np.ndarray:
    """
    Get the SNR of the given simulation IDs from the given table.

    Args:
        table (EventTable): Table with the SNR values
        simulation_ids (np.ndarray): Array with the simulation IDs

    Returns:
        np.ndarray: The SNR values of the given simulation IDs
    """
    
    # Result will be an array of the SNRs
    snr_array = np.zeros(len(simulation_ids))
    
    # Get the simulation IDs that are in this table
    simulation_id_xml = np.array(table["coinc_event_id"])
    
    # Iterate over the given simulation ids and fetch their SNR
    for i, sim_id in enumerate(simulation_ids):
        idx = np.where(simulation_id_xml == int(sim_id))[0][0]
        snr_array[i] = snr_xml[idx]
    
    return snr_array

In [65]:
snr_array = get_snr_array(table, data_dict["simulation_id"])
snr_array

array([20.71345432, 11.58412296, 31.55944294, 10.75187143, 10.835764  ,
       10.34055872, 10.48914612, 14.31458019, 13.58323679, 11.35493666,
       10.04501839, 20.23464631, 13.00715754, 10.19359192, 44.61218785,
       17.89609629, 10.38474232, 11.2444429 , 10.13507593, 10.42105646,
       15.48670876, 10.23033438, 55.42929873, 11.17614958, 12.04636938,
       13.22206652, 10.13372599, 15.12121194, 13.61569907, 13.98879268,
       10.49143778, 12.09639747, 11.17383179, 11.58139313, 15.94444037,
       11.62959458, 13.6450411 , 23.29264891, 10.03298407, 13.4044336 ,
       10.74093404, 59.65775812, 21.93262709, 11.58298514, 26.09528608,
       10.15430824, 13.68034927, 10.91762357, 10.53062619, 10.95658163,
       12.85539357, 17.30566715, 16.11623409, 18.61672576, 10.4006324 ,
       19.70476607, 16.11330413, 14.52496442, 27.2917565 , 10.02308949,
       12.6858746 , 23.12261274, 15.66958418, 16.06351711, 12.28981972,
       12.841571  , 12.45769719, 22.7830594 , 13.4711614 , 13.33